In [1]:
from pathlib import Path
import sys
from plotly import graph_objects as go
sys.path.append('/home/methodman/Projects/res-mod-dashboard/preprocessing')  # add the parent directory to sys.path
sys.path.append('/home/methodman/Projects/lib-main')  # add the parent directory to sys.path

from rolling_statistics import RollingStatistics
from config_manager import ConfigManager
from FileLogging.simple_logger import SimpleLogger
from data_reader import DataReader
from segmentation import Segmentation
from segment_feature_extraction import FeatureExtractor
from event_detection import EventDetector
from statistics_manager import StatisticsManager
from file_manager import FileManager

In [2]:
config_path = "/home/methodman/Projects/res-mod-dashboard/preprocessing/prep_config.yaml"
config_manager = ConfigManager(config_path)
config = config_manager.config
input_folder = Path(config['input_folder'])


In [3]:
for res_file in input_folder.glob(f'*{config["input_extension"]}'):
    if not res_file.is_file():
        continue
    else:
        break

df = DataReader().read_res_file(res_file, config['columns_to_process'])

In [4]:
df.head()


,epoch,sys,num,ppprtk1
0,2024-01-02,1.0,10.0,0.0448
1,2024-01-02,1.0,12.0,0.0570
2,2024-01-02,1.0,15.0,0.1380
3,2024-01-02,1.0,18.0,0.0969
4,2024-01-02,1.0,23.0,0.0485


In [5]:
grouped = df.groupby(['sys','num'])

for (sys,num), group_df in grouped:
    break

rolling_df = RollingStatistics().calculate(group_df, config)
rolling_df.tail()

,epoch,sys,num,ppprtk1,ppprtk1_diff,ppprtk1_rolling_mean,ppprtk1_rolling_std,ppprtk1_sg_filter
117643,2024-01-02 12:58:30,1.0,2.0,0.0101,0.0094,0.01155,0.008645,0.000460
117666,2024-01-02 12:58:35,1.0,2.0,0.0009,-0.0092,0.00891,0.007213,0.000217
117689,2024-01-02 12:58:40,1.0,2.0,0.0231,0.0222,0.00963,0.008270,0.000756
117712,2024-01-02 12:58:45,1.0,2.0,0.0006,-0.0225,0.00787,0.008115,0.000314
117735,2024-01-02 12:58:50,1.0,2.0,-0.0185,-0.0191,0.00426,0.010868,-0.000594


In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=rolling_df['epoch'],
    y=rolling_df['ppprtk1'],
    name = 'raw',
    mode = 'markers'
))
fig.add_trace(go.Scatter(
    x = rolling_df['epoch'],
    y = rolling_df['ppprtk1_rolling_mean']
))
fig.show()

In [22]:
events_dict = EventDetector(
    jump_threshold = 0.001,
    classification_tolerance = 0.001,
    stability_window = 2,
    shimmering_tolerance = 0.001,
    min_shimmering_window = 3,
    ).events_detection(rolling_df, config)
print(events_dict)

{}
